In [ ]:
from datasets import load_dataset
import openai, json
key6 = ''
openai.api_key = key6

In [ ]:
data = load_dataset("deepmind/code_contests")['test'] 
#UNKNOWN_LANGUAGE (0): 0
# PYTHON (1) (solutions written in PYTHON2): 1921
# CPP (2): 1585
# PYTHON3 (3): 1147
# JAVA (4): 4342
data

In [ ]:
# outputs = [] 
# for p in range( len(data) ):
#     dd = data[p]
#     for k in range(len( data[p]['solutions']['language'] )):
#         if data[p]['solutions']['language'][k] == 3:
#             outputs.append( json.dumps( { 'question': dd['description'], 'gold_completion': data[p]['solutions']['solution'][k] }) )
#             break

In [ ]:
# with open('python_codecontest.jsonl', "a") as f:
#     f.write("\n".join(outputs) + "\n")

In [ ]:
# get length of solutions
# len_k = []
# for p in range( 3, 4 ): #len(data) ):
#     for k in range(len( data[p]['solutions']['language'] )):
#         if data[p]['solutions']['language'][k] == 3:
#             len_k.append( len( data[p]['solutions']['solution'][k]) )
# sum(len_k)/ ( len(len_k) + 1e-6 )

In [ ]:
prompt_q = data[2]['description']
prompt_q

In [ ]:
def get_chatgpt_qa_response(prompt_text, temperature = 0.7, max_tokens=2048):
    messages = [{"role":"system", "content": "You are a helpful assistant that answers the question provided."},
                {"role":"user", "content": 'Provide me the Python3 codes for sovling the question: ' + prompt_text}]
    response = openai.ChatCompletion.create(
                model = "gpt-3.5-turbo",
                messages = messages,
                temperature = temperature,
                max_tokens = max_tokens,    
                )
    return response#['choices'][0]['message']['content']

In [ ]:
response = get_chatgpt_qa_response( prompt_text = chatgpt_python_codecontest[85]['question'] )

In [ ]:
response['choices'][0]['message']['content']

In [ ]:
with open( 'results/chatgpt_python_codecontest.jsonl', "r") as f:
    chatgpt_python_codecontest = [json.loads(x) for x in f.read().strip().split("\n") ]
len(chatgpt_python_codecontest)

In [ ]:
with open( 'results/regen_gpt-3.5-turbo_0.7_1.jsonl', "r") as f:
    regen = [json.loads(x) for x in f.read().strip().split("\n") ]
len(regen)

In [ ]:
for i, ins in enumerate(chatgpt_python_codecontest):
    if len( ins['gold_completion'].split(' ') ) >3000:
        print(i+1)

In [ ]:
regen[0]['human_gen_text']['choices']

In [ ]:
# chatgpt_python_codecontest[85]['gen_completion'] = response['choices'][0]['message']['content']

In [ ]:
chatgpt_python_codecontest[26]

In [ ]:
chatgpt_python_codecontest[5]

In [ ]:
for i,ins in enumerate(chatgpt_python_codecontest):
    if not ins['gen_completion'].startswith('def'):
        print(i )

# 26, 85: from import, 0, 53, 87, 97, 98, 110, 116: ''', 

In [ ]:
outputs = []
for ins in chatgpt_python_codecontest:
    outputs.append(json.dumps(ins))

with open( 'results/chatgpt_python_codecontest.jsonl', "w") as f:
    f.write("\n".join(outputs) + "\n")

In [ ]:
from utils import parse_code_snippet

for i,instance in enumerate( chatgpt_python_codecontest):
    print(i)
    pasrsed_codes = parse_code_snippet( prompt=instance['question'], raw_o=instance['gen_completion'] )
    chatgpt_python_codecontest[i]['pasrsed_codes'] = pasrsed_codes
    

In [ ]:
HUMANEVAL_EOS = ["\nclass", "\ndef", "\n#", "\n@", "\nprint", "\nif"]
NON_CODE_EOS = ["<|endoftext|>", "\n```", "\n</s>", "<|endofmask|>"]
EOS = HUMANEVAL_EOS + NON_CODE_EOS

def find_gen_func_sig(prompt):
    func_sig = ""
    for x in prompt.splitlines():
        if x.startswith("def ") and x.endswith(":"):
            # always pick the last one, since there could pre-defined functions.
            func_sig = x
    return func_sig

def remove_eos(gen):
    min_index = 1000
    for eos in EOS:
        if eos in gen:
            min_index = min(min_index, gen.index(eos))
    return gen[:min_index]

In [ ]:
raw_o = chatgpt_python_codecontest[2]['gen_completion'] # response['choices'][0]['message']['content'] #
prompt = chatgpt_python_codecontest[2]['question']
if "```" in raw_o:
    gen = raw_o.split("```")[1].strip()
    if gen.startswith("python"):
        gen = gen[len("python") :].strip()
    if gen.startswith(prompt.strip()):
        suf = gen.split(prompt.strip())[-1]
        suf = remove_eos(suf)
        gen = prompt.strip() + suf
    elif find_gen_func_sig(prompt) == '':
        gen = gen
    elif find_gen_func_sig(prompt) in gen:
        # same function sign is in the prompt
        sig = find_gen_func_sig(prompt)
        pre, suf = gen.split(sig)[0], gen.split(sig)[-1]
        suf = remove_eos(suf)
        gen = pre + sig + suf
    else:
        gen = f"# CANNOT PARSE CODE SNIPPET\n{gen}"
else:
    # cannot really handle parse just dump to file and maybe process later.
    gen = f"# CANNOT PARSE\n{raw_o}"
gen


In [ ]:
gen = raw_o.split("```")[1].strip()

In [ ]:
"```" in raw_o

In [ ]:
gen

In [ ]:
gen.startswith("python")

In [ ]:
gen.split('')